<a href="https://colab.research.google.com/github/Meghana-Merla/Fake-News-Detection/blob/main/News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project uses a publicly available dataset collected from an external source for academic and research purposes. The dataset contains news articles along with labels indicating whether the news is real or fake.


Importing Libraries

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#Printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Collection and Analysis


In [8]:
#Loading the dataset to DataFrame
news_dataset = pd.read_csv('/content/FA-KES-Dataset.csv',encoding='latin1')



In [9]:
#number of rows and colums
news_dataset.shape

(804, 7)

In [10]:

news_dataset.head()

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0
3,1914947534,Aleppo bomb blast kills six Syrian state TV,Wed 19 Apr 2017 Aleppo bomb blast kills six Sy...,nna,4/19/2017,aleppo,0
4,1914947535,29 Syria Rebels Dead in Fighting for Key Alepp...,Sun 10 Jul 2016 29 Syria Rebels Dead in Fighti...,nna,7/10/2016,aleppo,0


In [11]:
#Counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
unit_id,0
article_title,0
article_content,0
source,0
date,0
location,0
labels,0


In [12]:
#Replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [13]:
#Separating the data & label
X = news_dataset.drop(columns='labels', axis=1)
Y = news_dataset['labels']

In [14]:
print(X)
print(Y)

        unit_id                                      article_title  \
0    1914947530  Syria attack symptoms consistent with nerve ag...   
1    1914947532  Homs governor says U.S. attack caused deaths b...   
2    1914947533    Death toll from Aleppo bomb attack at least 112   
3    1914947534        Aleppo bomb blast kills six Syrian state TV   
4    1914947535  29 Syria Rebels Dead in Fighting for Key Alepp...   
..          ...                                                ...   
799  1965511221    Turkish Bombardment Kills 20 Civilians in Syria   
800  1965511222    Martyrs as Terrorists Shell Aleppos Salah Eddin   
801  1965511224  Chemical Attack Kills Five Syrians in Aleppo SANA   
802  1965511226  5 Killed as Russian Military Chopper Shot down...   
803  1965511231  Syrian Army Kills 48 ISIL Terrorists in Deir E...   

                                       article_content source       date  \
0    Wed 05 Apr 2017 Syria attack symptoms consiste...    nna   4/5/2017   
1    Fr

Stemming:

Stemming is the process of reducing a word to its Root word

In [15]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [17]:
news_dataset['article_content'] = news_dataset['article_content'].apply(stemming)

In [18]:
print(news_dataset['article_content'])

0      wed apr syria attack symptom consist nerv agen...
1      fri apr hom governor say u attack caus death d...
2      sun apr death toll aleppo bomb attack least de...
3      wed apr aleppo bomb blast kill six syrian stat...
4      sun jul syria rebel dead fight key aleppo road...
                             ...                        
799    turkish bombard kill civilian syria turkish sh...
800    martyr terrorist shell aleppo salah eddin seve...
801    chemic attack kill five syrian aleppo sana lea...
802    kill russian militari chopper shot syria five ...
803    april syrian armi kill isil terrorist deir ezz...
Name: article_content, Length: 804, dtype: object


In [19]:
#Separating the data and label
X = news_dataset['article_content'].values
Y = news_dataset['labels'].values

In [20]:
print(X)

['wed apr syria attack symptom consist nerv agent use victim suspect chemic attack syria appear show symptom consist reaction nerv agent world health organ said wednesday case appear show addit sign consist exposur organophosphoru chemic categori chemic includ nerv agent said statement put death toll least unit state said death caus sarin nerv ga drop syrian aircraft russia said believ poison ga leak rebel chemic weapon depot struck syrian bomb sarin organophosporu compound nerv agent chlorin mustard ga also believ use past syria russian defenc ministri spokesman say agent use attack said rebel use chemic weapon aleppo last year said like kind chemic use attack suffer appar extern injuri die rapid onset similar symptom includ acut respiratori distress said expert turkey give guidanc overwhelm health worker idlib diagnosi treatment patient medicin atropin antidot type chemic exposur steroid symptomat treatment sent u n commiss inquiri human right syria previous said forc loyal syrian pr

In [21]:
print(Y)

[0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 0 0
 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0
 1 1 0 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 1 1 0 0
 1 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 1 1 1 0 0 0 1 1 0 1 0 1
 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 1 0 1 1
 0 1 1 0 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0
 1 0 0 0 1 1 1 0 1 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 0 0 1 1
 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1
 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1
 1 1 0 1 1 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 0
 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 0
 0 1 0 0 1 1 0 0 0 1 0 1 

In [22]:
Y.shape

(804,)

In [23]:
#Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [24]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 96418 stored elements and shape (804, 6894)>
  Coords	Values
  (0, 86)	0.0773133752433779
  (0, 93)	0.04431036073814104
  (0, 141)	0.30975301520720666
  (0, 148)	0.047098867769827905
  (0, 165)	0.04120539118174339
  (0, 196)	0.01624091966462088
  (0, 217)	0.021372240427278352
  (0, 252)	0.032544945493671845
  (0, 276)	0.020388205043728028
  (0, 284)	0.07457694095103234
  (0, 364)	0.08084125262864728
  (0, 380)	0.061950603041441336
  (0, 384)	0.10362590273590017
  (0, 394)	0.04970553247355266
  (0, 488)	0.07372093158653079
  (0, 521)	0.08581351653518718
  (0, 525)	0.10433199892314882
  (0, 538)	0.03735897350554646
  (0, 634)	0.051069504859539115
  (0, 653)	0.061950603041441336
  (0, 677)	0.029096958369380654
  (0, 735)	0.08624536101002035
  (0, 819)	0.02404480420186257
  (0, 879)	0.048478197843092145
  (0, 985)	0.056407858987417694
  :	:
  (803, 5244)	0.12493612927842798
  (803, 5262)	0.026252234626668224
  (803, 5324)	0.1102

Splitting the dataset to training & test data

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression


In [26]:
model = LogisticRegression()

In [27]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Getting Accuracy Score

In [28]:
#Accuracy Score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [29]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.880248833592535


In [30]:
#Accuracy Score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [31]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.5527950310559007


Making a Predictive System

In [32]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [33]:
print(Y_test[3])

1
